In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import bioio_ome_zarr
import matplotlib.pyplot as plt
from pathlib import Path
import os
import zarr
import tifffile
import gc  # for freeing memory
from skimage import exposure
from skimage.transform import resize
from fuse_n5_xml_to_tif import pix_size

In [3]:
# ----- File Paths -----
# base_path per OS
# macOS
# base_path = Path("/Users/thomasnegrello/mnt/tnegrell_ista/siegegrp/ThNe/development_retina/raw_images_n5/")
# Ubuntu
# base_path = Path("/mnt/gdrive/ThNe/development_retina/raw_images_n5/")
# Windows
# base_path = Path("Z:/siegegrp/ThNe/development_retina/")
#debian
base_path = Path("/run/user/1009/gvfs/smb-share:server=fs.ista.ac.at,share=drives/tnegrell/group/siegegrp/ThNe/development_retina/")
fuse_folder = Path("trila_fuse_mg/P21/F/L/")
file_name = "microglia_Age_P21_Sex_F_Side_L_Animal_1"

file_path = base_path / fuse_folder / file_name

zarr_path = file_path.with_suffix(".zarr")


In [4]:
dapi_path = 'dapi'
mg_path = 'microglia'
blood_vessels = 'blood'

In [5]:

# Open the Zarr file (can be a directory or .zarr file)
z = zarr.open(zarr_path, mode='r')

# List the groups and arrays inside
z.tree()

Tree(nodes=(Node(disabled=True, name='/', nodes=(Node(disabled=True, icon='table', name='0 (1, 1, 54, 7486, 74…

In [7]:
# Open the Zarr
z = zarr.open(zarr_path, mode='r')
arr = z['0'] 

In [8]:
arr.shape

(1, 1, 54, 7486, 7481)

In [ ]:
# Map channel index to subfolder
channel_map = {
    0: dapi_path,
    1: mg_path,
    2: blood_vessels
}

# Open the Zarr
z = zarr.open(zarr_path, mode='r')
arr = z['0']  # shape: (1, 3, Z, Y, X)

# Process each channel one by one
for ch_idx, subfolder in channel_map.items():
    print(f"Processing channel {ch_idx} → {subfolder}")
    
    # Load data
    data = arr[0, ch_idx, :, :, :]  # shape: (Z, Y, X)
    
    # Get dimensions
    z_max_pix, y_max_pix, x_max_pix = data.shape

    # Build output path
    tif_filename = f"{subfolder}_Age_P15_Sex_F_Side_L_Animal_1.tif"
    out_path = os.path.join(base_path, subfolder, "P15", "F", "L")
    os.makedirs(out_path, exist_ok=True)
    full_tif_path = os.path.join(out_path, tif_filename)

    # Metadata with physical size info
    metadata = {
        # Physical dimensions in microns
        "ExtendMinX": 0,
        "ExtendMaxX": x_max_pix * pix_size['x'],
        "ExtendMinY": 0,
        "ExtendMaxY": y_max_pix * pix_size['y'],
        "ExtendMinZ": 0,
        "ExtendMaxZ": z_max_pix * pix_size['z'],
        # Dimensions in pixels
        "SizeX": x_max_pix,
        "SizeY": y_max_pix,
        "SizeZ": z_max_pix,
    }

    # Save as TIFF
    tifffile.imwrite(
        full_tif_path,
        data,
        metadata=metadata,
        dtype=data.dtype
    )
    print(f"Saved {full_tif_path}")

    # Free memory
    del data
    gc.collect()
